In [1]:
# imports
import numpy as np
from PIL import Image
from numpy import asarray
from sklearn.model_selection import train_test_split
import os
import seaborn as sn
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD

In [2]:
NumberModlesToTest = 10

In [3]:
# mountin google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# change directory of dataset based on the drive
DIRECTORY = "/content/drive/MyDrive/MaskDataset"
CATEGORIES = ["with_mask", "without_mask"]

In [5]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        # load the image
        image = Image.open(img_path).resize((224, 224))
       
        # convert image to numpy array
        image_data = asarray(image)
        data.append(image_data.reshape((224,224,3)))

        # perform one-hot encoding on the labels
        if category == "with_mask":
            labels.append(0)
        else:
            labels.append(1)


data = np.array(data, dtype="float32")
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

In [6]:
def test_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_result = []
    for p in y_pred:
        y_result.append(int(p >= 0.5))
    
    y_actual = y_test
    
    print(classification_report(y_actual, y_result))
    
    cm = tf.math.confusion_matrix(labels = y_actual, predictions = y_result)

    plt.figure(figsize = (10, 8))
    sn.heatmap(cm, annot = True, fmt = 'd')
    plt.xlabel('Predicted')
    plt.ylabel('Truth')

In [7]:
def Model(layers, dense_nodes, drop, activation, epochs, lr , X_train, X_test, y_train, y_test, validation_split,batch_size):
    # define model
    model = Sequential()
    model.add(Conv2D(dense_nodes, (3, 3), activation='relu', input_shape= (224, 224, 3)))
    model.add(MaxPooling2D(2, 2))

    for i in range(1,layers):
      model.add(Conv2D(dense_nodes/(2**i), (3, 3), activation='relu'))
      model.add(MaxPooling2D(2, 2))
      model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dropout(drop))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation=activation))
 
    opt = SGD(lr=lr)

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.summary()
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                        validation_split=validation_split)
    test_model(model, X_test,y_test )

    return model

In [ ]:
nodes = 256
drop_rate = 0.40
activation = 'sigmoid'
epochs = 20
lr = 0.0001
validation_split = 0.2
batch_size = 100

for i in range(3,NumberModlesToTest):
    name = "model_weights" + str(i) + ".h5"
    print(name)
    model = Model(i, nodes,drop_rate,activation,epochs,lr,X_train, X_test, y_train, y_test,validation_split,batch_size)
    model.save(name)

model_weights3.h5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 256)     7168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 256)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 128)     295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 54, 54, 128)      512       
 ormalization)                                                   
                                      

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/20


In [ ]:
from keras.models import load_model
def Load(model_weights_file):

        # load weights into the new model
        model = load_model(model_weights_file)
        return model

In [ ]:
!pip install lime

from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.segmentation import mark_boundaries

explainer = lime_image.LimeImageExplainer() 
segmenter = SegmentationAlgorithm('quickshift', kernel_size=1, max_dist=200, ratio=0.2)

In [ ]:
for i in range(3,NumberModlesToTest):
    name = "model_weights" + str(i) + ".h5"
    model=Load(name)
    image = X_test[0]
    explanation = explainer.explain_instance(image.astype('double'), classifier_fn = model.predict)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], 
                                            positive_only=True, 
                                            num_features=20, 
                                            hide_rest=True)

    plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
    plt.show()